# CNN-text-classification
- 调试须知：python中的函数不到调用不会检查 即便有拼写等错误

## prepocessiong
- 分词
- 词语 -> id
    - 由于：matrix -> [|V|, embed_size]
    - 词表 从而得到词语的id表示(词语A -> id(5))
- label -> id

In [1]:
import sys
import os
import jieba
import tensorflow as tf
import numpy as np
import math

In [2]:
# input files
train_file = './text_classification/cnews.train.txt'
val_file = './text_classification/cnews.val.txt'
test_file = './text_classification/cnews.test.txt'
#output files
seg_train_file = './text_classification/cnews.train.seg.txt'
seg_val_file = './text_classification/cnews.val.seg.txt'
seg_test_file = './text_classification/cnews.test.seg.txt'

vocab_file = './text_classification/cnews.vocab.txt'
category_file = './text_classification/cnews.category.txt'

### file read demo

In [3]:

'''通过bytes-like对象读取
with open(test_file,'rb') as f:
    lines = f.readlines()
label,content = lines[0].decode('utf-8').strip('\r\n').split('\t')
'''
'''直接在读取的时候指定读取的编码'''
with open(test_file,'r',encoding='utf-8') as f:
    lines = f.readlines()
label,content = lines[0].strip('\r\n').split('\t')
'''如果以上两种都不用 报错 'gbk' codec can't decode意思是无法通过gbk进行解码
原因是文本本身不是gbk 所以无法用默认的gbk方式解码 
所以最直接的通过相应格式进行文件读取
/使用读取字节文件方式读取 之后再对读取内容进行解码
'''
words = jieba.cut(content)
print(content)
print(' '.join(words))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\gmm\AppData\Local\Temp\jieba.cache


鲍勃库西奖归谁属？ NCAA最强控卫是坎巴还是弗神新浪体育讯如今，本赛季的NCAA进入到了末段，各项奖项的评选结果也即将出炉，其中评选最佳控卫的鲍勃-库西奖就将在下周最终四强战时公布，鲍勃-库西奖是由奈史密斯篮球名人堂提供，旨在奖励年度最佳大学控卫。最终获奖的球员也即将在以下几名热门人选中产生。〈〈〈 NCAA疯狂三月专题主页上线，点击链接查看精彩内容吉梅尔-弗雷戴特，杨百翰大学“弗神”吉梅尔-弗雷戴特一直都备受关注，他不仅仅是一名射手，他会用“终结对手脚踝”一样的变向过掉面前的防守者，并且他可以用任意一支手完成得分，如果他被犯规了，可以提前把这两份划入他的帐下了，因为他是一名命中率高达90%的罚球手。弗雷戴特具有所有伟大控卫都具备的一点特质，他是一位赢家也是一位领导者。“他整个赛季至始至终的稳定领导着球队前进，这是无可比拟的。”杨百翰大学主教练戴夫-罗斯称赞道，“他的得分能力毋庸置疑，但是我认为他带领球队获胜的能力才是他最重要的控卫职责。我们在主场之外的比赛(客场或中立场)共取胜19场，他都表现的很棒。”弗雷戴特能否在NBA取得成功？当然，但是有很多专业人士比我们更有资格去做出这样的判断。“我喜爱他。”凯尔特人主教练多克-里弗斯说道，“他很棒，我看过ESPN的片段剪辑，从剪辑来看，他是个超级巨星，我认为他很成为一名优秀的NBA球员。”诺兰-史密斯，杜克大学当赛季初，球队宣布大一天才控卫凯瑞-厄尔文因脚趾的伤病缺席赛季大部分比赛后，诺兰-史密斯便开始接管球权，他在进攻端上足发条，在ACC联盟(杜克大学所在分区)的得分榜上名列前茅，但同时他在分区助攻榜上也占据头名，这在众强林立的ACC联盟前无古人。“我不认为全美有其他的球员能在凯瑞-厄尔文受伤后，如此好的接管球队，并且之前毫无准备。”杜克主教练迈克-沙舍夫斯基赞扬道，“他会将比赛带入自己的节奏，得分，组织，领导球队，无所不能。而且他现在是攻防俱佳，对持球人的防守很有提高。总之他拥有了辉煌的赛季。”坎巴-沃克，康涅狄格大学坎巴-沃克带领康涅狄格在赛季初的毛伊岛邀请赛一路力克密歇根州大和肯塔基等队夺冠，他场均30分4助攻得到最佳球员。在大东赛区锦标赛和全国锦标赛中，他场均27.1分，6.1个篮板，5.1次助攻，依旧如此给力。他以疯狂的表现开始这个赛季，也将以疯狂的表现结束这个赛季。“我们在全国锦标赛中前进着，并且之前曾经

Loading model cost 0.688 seconds.
Prefix dict has been built succesfully.


鲍勃 库西 奖归 谁 属 ？   NCAA 最强 控卫 是 坎巴 还是 弗神 新浪 体育讯 如今 ， 本赛季 的 NCAA 进入 到 了 末段 ， 各项 奖项 的 评选 结果 也 即将 出炉 ， 其中 评选 最佳 控卫 的 鲍勃 - 库西 奖 就 将 在 下周 最终 四强 战时 公布 ， 鲍勃 - 库西 奖是 由奈 史密斯 篮球 名人堂 提供 ， 旨在 奖励 年度 最佳 大学 控卫 。 最终 获奖 的 球员 也 即将 在 以下 几名 热门 人选 中 产生 。 〈 〈 〈   NCAA 疯狂 三月 专题 主页 上线 ， 点击 链接 查看 精彩内容 吉梅尔 - 弗雷 戴特 ， 杨百翰 大学 “ 弗神 ” 吉梅尔 - 弗雷 戴特 一直 都 备受 关注 ， 他 不仅仅 是 一名 射手 ， 他会用 “ 终结 对手 脚踝 ” 一样 的 变向 过 掉 面前 的 防守 者 ， 并且 他 可以 用 任意 一支 手 完成 得分 ， 如果 他 被 犯规 了 ， 可以 提前 把 这 两份 划入 他 的 帐 下 了 ， 因为 他 是 一名 命中率 高达 90% 的 罚球 手 。 弗雷 戴特 具有 所有 伟大 控卫 都 具备 的 一点 特质 ， 他 是 一位 赢家 也 是 一位 领导者 。 “ 他 整个 赛季 至始 至终 的 稳定 领导 着 球队 前进 ， 这是 无可比拟 的 。 ” 杨百翰 大学 主教练 戴夫 - 罗斯 称赞 道 ， “ 他 的 得分 能力 毋庸置疑 ， 但是 我 认为 他 带领 球队 获胜 的 能力 才 是 他 最 重要 的 控卫 职责 。 我们 在 主场 之外 的 比赛 ( 客场 或 中 立场 ) 共 取胜 19 场 ， 他 都 表现 的 很棒 。 ” 弗雷 戴特 能否 在 NBA 取得成功 ？ 当然 ， 但是 有 很多 专业人士 比 我们 更 有 资格 去 做出 这样 的 判断 。 “ 我 喜爱 他 。 ” 凯尔特人 主教练 多克 - 里 弗斯 说道 ， “ 他 很棒 ， 我 看过 ESPN 的 片段 剪辑 ， 从 剪辑 来看 ， 他 是 个 超级 巨星 ， 我 认为 他 很 成为 一名 优秀 的 NBA 球员 。 ” 诺兰 - 史密斯 ， 杜克大学 当 赛季 初 ， 球队 宣布 大 一天 才 控卫凯瑞 - 厄尔 文因 脚趾 的 伤病 缺席 赛季 大部分 比赛 后 ， 

### 断句 分词

In [4]:
def segment_file(input_file,output_file):
    '''segment the sentences in each line from the inputfile'''
    with open(input_file,'r',encoding='utf-8') as f:
        lines = f.readlines()
    with open(output_file,'w',encoding='utf-8') as f:
        for line in lines:
            label,content = line.strip('\r\n').split('\t')
            words = jieba.cut(content)
            content_by_word = ''
            for word in words:
                word.strip(' ')
                if word != '':
                    content_by_word += word+' '
            out_line = '%s\t%s\n'%(label,content_by_word.strip(' '))
            f.write(out_line)
segment_file(train_file,seg_train_file)
segment_file(val_file,seg_val_file)
segment_file(test_file,seg_test_file)

### 生成词表 类表
- 行号索引是id
- 后面接上次数 便于后续处理 以及观察分布信息

In [5]:
def generate_vocab_file(input_seg_file,output_vocab_file):
    with open(input_seg_file,'r',encoding='utf-8') as f:
        lines = f.readlines()
    word_dict = {} # 统计词频
    for line in lines:
        label,content = line.strip('\r\n').split('\t')
        for word in content.split():
            # 如果没有这个key设置为0
            word_dict.setdefault(word,0)
            word_dict[word] += 1 
    # 按照频率进行排序 使用lambda函数对于元祖列表排序制定关键字
    sorted_word_dict = sorted(word_dict.items(),
                             key=lambda word:word[1],reverse=True)
    with open(output_vocab_file,'w',encoding='utf-8') as f:
        '''
        写入未知字符标识
        1:对于舍掉不常用的词可以再train中使用这个作为id
        2:对于val / test中出现的train中没有的可以用这个作为id
        3:对于不足threshold长度的sentence可以使用这个进行补齐
        '''
        f.write('<UNK>\t100000000\n')
        for item in sorted_word_dict:
            f.write('%s\t%d\n' % item)
generate_vocab_file(seg_train_file,vocab_file)

In [6]:
def generate_category_file(input_file,output_category_file):
    with open(input_file,'r',encoding='utf-8') as f:
        lines = f.readlines()
    category_dict = {} # 统计词频
    for line in lines:
        label,content = line.strip('\r\n').split('\t')
        category_dict.setdefault(label,0)
        category_dict[label] += 1 
    # 按照频率进行排序 使用lambda函数对于元祖列表排序制定关键字
    sorted_category_dict = sorted(category_dict.items(),
                             key=lambda c:c[1],reverse=True)
    with open(output_category_file,'w',encoding='utf-8') as f:
        for item in sorted_category_dict:
            print('%s\t%d\n' % item)
            f.write('%s\n' % item[0])
            
generate_category_file(train_file,category_file)

体育	5000

娱乐	5000

家居	5000

房产	5000

教育	5000

时尚	5000

时政	5000

游戏	5000

科技	5000

财经	5000



## 数据集封装+超参数定义
- next_batch api 
    - read seg file get sentences id and category id by vocab and category class
- 词表封装
    - wordtoid api
- 类别表封装
    - categorytoid api

### 超参数定义(by HParams+tf logging output)

In [9]:
tf.logging.set_verbosity(tf.logging.INFO)

def get_default_param():
    """
    return HParams object to get hyper parameters
    可以通过构造方法传入参数 通过对象直接访问参数
    """
    '''cnn 一维卷积特有的
    - 指定输出通道数目 由filter数目决定
    - 可定义多个 一维卷积kernel规模
    '''
    return tf.contrib.training.HParams(
        num_embedding_size = 16,#embedding length
        num_timesteps = 50, #必须有
        
        num_filters = [64,],
        num_kernel_size = [3,],
        num_fc_nodes = 32,
        batch_size = 100,
        learning_rate = 1e-3,
        num_word_threshold = 20,#超过这个长度额截断 不足补齐
        # 防止梯度爆炸 梯度消失可以用lstm特性避免一部分
    )
hps = get_default_param()

# 定义需要用到的w文件
train_seg_file = './text_classification/cnews.train.seg.txt'
val_seg_file = './text_classification/cnews.val.seg.txt'
test_seg_file = './text_classification/cnews.test.seg.txt'
vocab_file = './text_classification/cnews.vocab.txt'
category_file = './text_classification/cnews.category.txt'

output_folder = './text_classification/run_text_rnn'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    

### 词表封装
- 实现sentencetoid api

In [10]:
class Vocab:
    def __init__(self,filename,num_word_threshold):
        self._num_word_threshold = num_word_threshold
        self._unk = -1
        self._word_to_id_dict = {}
        self._read_file(filename)
    def _read_file(self,filename):
        with open(filename,'r',encoding='utf-8') as f:
            lines = f.readlines()
        for line in lines:
            word,freq = line.strip('\r\n').split('\t')
            freq = int(freq)
            if freq < self._num_word_threshold:
                continue
            nowid = len(self._word_to_id_dict)
            if word == '<UNK>':
                self._unk = nowid
            self._word_to_id_dict[word] = nowid
    def _word_to_id(self,word):
        '''没有的返回unk的id'''
        return self._word_to_id_dict.get(word,self._unk)
    
    # 使用property 可以像获取公开成员方式访问函数
    @property
    def unk(self):
        return self._unk
    
    def get_size(self):
        return len(self._word_to_id_dict)
    def sentence_to_id(self,seg_sentence):
        word_ids = [self._word_to_id(w) for w in seg_sentence.split()]
        return word_ids
    
class Category:
    def __init__(self,filename):
        self._category_to_id_dict = {}
        self._read_file(filename)
        
    def _read_file(self,filename):
        with open(filename,'r',encoding='utf-8') as f:
            lines = f.readlines()
        for line in lines:
            category = line.strip('\r\n')
            nowid = len(self._category_to_id_dict)
            self._category_to_id_dict[category] = nowid
    
    def get_size(self):
        return len(self._category_to_id_dict)
    
    def category_to_id(self,category_name):
        if not category_name in self._category_to_id_dict.keys():
            raise Exception(
                "%s is not in our category list" % category_name)
        return self._category_to_id_dict[category_name]
    
vocab = Vocab(vocab_file,hps.num_word_threshold)
'''size 在后续的网络搭建中会用到'''
vocab_size = vocab.get_size()

category = Category(category_file)
num_classes = category.get_size()

tf.logging.info('classes_num is:%d\n'%num_classes)
demo_str = '体育'
tf.logging.info('label is:%s\t id is:%s\n' % \
                (demo_str,category.category_to_id(demo_str)))

INFO:tensorflow:classes_num is:10

INFO:tensorflow:label is:体育	 id is:0



### dataset

In [11]:
class TextClassificationDataSet:
    def __init__(self,filename,vocab,category,num_timesteps,need_shuffle=True):
        """
        提供next_batch 并且需要通过num_timesteps 得到batch中的语句词语数目
        便于对齐
        """
        self._vocab = vocab
        self._category = category
        self._num_timesteps = num_timesteps
        self._need_shuffle = need_shuffle
        '''data is id matrix,label is id vector'''
        self._datas = []
        self._labels = []
        self._indicator = 0
        self._parse_file(filename)
    def _parse_file(self,filename):
        tf.logging.info('loading data from %s' % filename)
        with open(filename,'r',encoding='utf-8') as f:
            lines = f.readlines()
        for line in lines:
            label,content = line.strip('\r\n').split('\t')
            id_label = self._category.category_to_id(label)
            id_content = self._vocab.sentence_to_id(content)
            '''进行对齐长度处理(尽可能少用if)'''
            id_content = id_content[0:self._num_timesteps]
            padding_num = self._num_timesteps - len(id_content)
            id_content = id_content + [self._vocab.unk for i in range(padding_num)]
            self._datas.append(id_content)
            self._labels.append(id_label)
        '''切记对于从文本中读取的数字是str 需要转换成int类型 ?在tf中可以自动转换为float32'''
        self._datas = np.asarray(self._datas,dtype=np.int32)
        self._labels = np.asarray(self._labels,dtype=np.int32)
        '''首次生成需要random shuffle'''
        self._random_shuffle()
        
    def next_batch(self,batch_size):
        # 默认参数的值不能是init声明的成员变量吗??
        if batch_size > len(self._datas):
            raise Execption("batch_size: %d is too large" % batch_size)        
        end_indicator = self._indicator + batch_size
        if end_indicator > len(self._datas):
            if self._need_shuffle:
                self._random_shuffle()
            self._indicator = 0
            end_indicator = batch_size
        
        batch_datas = self._datas[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_datas,batch_labels
    
    def _random_shuffle(self):
        idx = np.random.permutation(len(self._datas))
        self._datas = self._datas[idx]
        self._labels = self._labels[idx]

train_dataset = TextClassificationDataSet(train_seg_file,vocab,
                                          category,hps.num_timesteps)
val_dataset = TextClassificationDataSet(val_seg_file,vocab,category,
                                        hps.num_timesteps,False)
test_dataset = TextClassificationDataSet(test_seg_file,vocab,category,
                                         hps.num_timesteps,False)

print(train_dataset.next_batch(1))
print(val_dataset.next_batch(1))
print(test_dataset.next_batch(1))


INFO:tensorflow:loading data from ./text_classification/cnews.train.seg.txt
INFO:tensorflow:loading data from ./text_classification/cnews.val.seg.txt
INFO:tensorflow:loading data from ./text_classification/cnews.test.seg.txt
(array([[ 5053,  1337,    22,     0,  5730,  2499,  2897,    45,   688,
           52,  1296,   660,    78,   103,    23,  1166,    44,     1,
          225,  5053,  4314,    22,  5049, 20815,   970,     2,  4509,
         5730,    50, 21777,  7485,     1,   508,  5544,  4057, 12425,
            1,  1067,   562,  6199,     2,     8, 11164,  2942,     9,
            3,  1199,  2942,  3033,  8393]]), array([4]))
(array([[19061, 13982,    43, 45285, 19061,   273,  3545,  7416,    76,
            0,  2144,     1,  4020,   579,    15,   602,   883, 26042,
           57,     4, 19061,   527,   271, 15235,    14,     1,   161,
          987,    22,   113,    94,    23,    75,  2189, 10031,     3,
           43,  2058,  7847,     1,  1155,  7387,   651,   248,   704,
     

## create model
- 搭建cnn text classification model
- 效果：更快收敛 每一步训练更快 效果也会更好

In [14]:
'''edition 2 手动实现lstmcell'''
def create_model(hps,vocab_size,num_classes):
    num_timesteps = hps.num_timesteps
    batch_size = hps.batch_size   
    inputs = tf.placeholder(tf.int32,(batch_size,num_timesteps))
    outputs = tf.placeholder(tf.int32,(batch_size,))
    keep_prob = tf.placeholder(tf.float32,name='dropout_keep_prob')
    
    '''input_layer'''
    embedding_initializer = tf.random_uniform_initializer(-1.0,1.0)
    with tf.variable_scope('embedding',initializer=embedding_initializer):
        embeddings = tf.get_variable('vocab_embedding_matrix',
                                    [vocab_size,hps.num_embedding_size],
                                    tf.float32)

        embed_inputs = tf.nn.embedding_lookup(embeddings,inputs)

    '''cnn layer'''
    scale = 1.0/math.sqrt(hps.num_embedding_size+hps.num_filters[0])/3.0
    cnn_initializer = tf.random_uniform_initializer(-scale,scale)
    with tf.variable_scope('cnn_layer',initializer=cnn_initializer):
        '''
        如果使用一种卷积kernel长度
        embed_inputs: [batch_size, timesteps, embed_size]
        conv1d: [batch_size, timesteps, num_filters]
        如果多种长度kernel 处理方法类似
        '''
        conv1d = tf.layers.conv1d(embed_inputs,hps.num_filters[0],
                                 hps.num_kernel_size[0],activation=tf.nn.relu)
        '''时间维度pooling'''
        global_maxpooling = tf.reduce_max(conv1d,axis=[1])
        
    '''fc layer + output layer'''
    fc_initializer = tf.uniform_unit_scaling_initializer(factor=1.0)
    with tf.variable_scope('fc_layer',initializer=fc_initializer):
        fc1 = tf.layers.dense(global_maxpooling,hps.num_fc_nodes,
                             activation=tf.nn.relu,name='fc1')
        droped_fc1 = tf.contrib.layers.dropout(fc1,keep_prob)
        logits = tf.layers.dense(droped_fc1,num_classes,name='fc2')
        
    with tf.name_scope('metrics'):
        loss = tf.losses.sparse_softmax_cross_entropy(
            logits=logits,labels=outputs)
        y_predict = tf.argmax(tf.nn.softmax(logits),
                              axis=1,output_type=tf.int32) #小心类型 才能equal
        correct_pred = tf.equal(outputs,y_predict)
        accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    global_step = tf.Variable(tf.zeros([],tf.int64),name='global_step',
                             trainable=False)

    with tf.name_scope('train_op'):
        train_op = tf.train.AdamOptimizer(hps.learning_rate).minimize(
            loss, global_step=global_step)
    return ((inputs,outputs,keep_prob),
           (loss,accuracy),
           (train_op,global_step))    

        

## train process

In [15]:
tf.reset_default_graph()
placeholder,metrics,others = create_model(hps,vocab_size,num_classes)

inputs,outputs,keep_prob = placeholder
loss,accuracy = metrics
train_op,global_step = others

In [16]:
init_op = tf.global_variables_initializer()
train_keep_prob_val = 0.7
test_keep_prob_val = 1.0
num_train_steps = 10000
# Train: 99.7%
# Valid: 92.7%
# Test:  93.2%
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(num_train_steps):
        batch_datas,batch_labels = train_dataset.next_batch(hps.batch_size)
        output_val = sess.run([loss,accuracy,train_op,global_step],
                             feed_dict={
                                 inputs:batch_datas,
                                 outputs:batch_labels,
                                 keep_prob:train_keep_prob_val
                             })
        loss_val,accuracy_val,_,global_step_val = output_val
        if global_step_val % 50 == 0:
            tf.logging.info("Step: %5d, loss: %3.3f, accuracy: %3.3f"
                            % (global_step_val, loss_val, accuracy_val))

INFO:tensorflow:Step:     0, loss: 2.339, accuracy: 0.110
INFO:tensorflow:Step:    50, loss: 2.256, accuracy: 0.220
INFO:tensorflow:Step:   100, loss: 2.069, accuracy: 0.330
INFO:tensorflow:Step:   150, loss: 1.601, accuracy: 0.470
INFO:tensorflow:Step:   200, loss: 1.409, accuracy: 0.530
INFO:tensorflow:Step:   250, loss: 1.254, accuracy: 0.600
INFO:tensorflow:Step:   300, loss: 1.227, accuracy: 0.580
INFO:tensorflow:Step:   350, loss: 1.082, accuracy: 0.650
INFO:tensorflow:Step:   400, loss: 0.919, accuracy: 0.730
INFO:tensorflow:Step:   450, loss: 0.886, accuracy: 0.710
INFO:tensorflow:Step:   500, loss: 0.812, accuracy: 0.710
INFO:tensorflow:Step:   550, loss: 0.783, accuracy: 0.760
INFO:tensorflow:Step:   600, loss: 0.853, accuracy: 0.690
INFO:tensorflow:Step:   650, loss: 0.597, accuracy: 0.810
INFO:tensorflow:Step:   700, loss: 0.775, accuracy: 0.770
INFO:tensorflow:Step:   750, loss: 0.680, accuracy: 0.800
INFO:tensorflow:Step:   800, loss: 0.802, accuracy: 0.730
INFO:tensorflo

KeyboardInterrupt: 

## 总结
- 使用cnn架构虽然无法处理变长序列问题 但是可以更快 因为cnn并行计算远快于rnn
- 目前为止已学到的模型
    - rnn
        - 单向lstm取最后输出
        - 双向并对考虑所有输出
        - hiarachical attention lstm 加入了层级关系 并且引入了注意力模型
    - cnn(虽然应用场景受到限制 但是更快表现甚至更好)
        - cnn 文本分类器(一维卷积)
    - rnn+cnn
        - 用rnn接受边长输入 得到输出作为embedding(取代原来直接就是输入的imbedding)
        - 再用cnn进行处理 之后pooling + fc得到结果
        - 结合二者优点 拓宽cnn使用场景
